In [9]:
import pandas as pd
import sys
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, KFold
from sentence_transformers import SentenceTransformer
import numpy as np

sys.path.append("../")
import src.data_utils as d_u
import src.feats_generation as f_g

Сначала с помощью подбора гиперпараметров посмотрим, какое наилучшее решение мы можем получить, используя линейную регрессию с L2 регуляризацией

In [5]:
# Получим предобработанный датасет

df = pd.read_csv("../data/raw/АВСОФТ_тест_ML_приложение.csv")
df.drop(columns=["commit_hash"], inplace=True)
df = pd.concat([df, pd.get_dummies(df.repository_name)], axis=1)
df.drop(columns=["repository_name"], inplace=True)
df = f_g.ohe(df, "commit_author")
df["commit_date"] = pd.to_datetime(df.commit_date)

df = f_g.encode_work_days(df)
df = f_g.encode_work_hours(df)

if "commit_date" in df.columns:
    df.drop(columns=["commit_date"], inplace=True)

X = df.drop(columns=["commit_message", "bugs"])
y = df.bugs

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

msg_embs = model.encode(df.commit_message.values)

X = np.concatenate((X.to_numpy(), msg_embs), axis=1)

In [17]:
# конфигурация для поиска наилучшего решения

ridge_config = {
    "alpha": [0.5, 1.0, 1.5, 5.0, 10],
    "solver": ["auto", "svd", "cholesky", "sag"],
    "random_state": [42],
    "tol": [1e-2, 1e-3, 1e-4]
}


In [20]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

model_ridge = Ridge(random_state=42)
gs_ridge = GridSearchCV(model_ridge, ridge_config, cv=cv, scoring="neg_mean_squared_error", verbose=1)
search = gs_ridge.fit(X, y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [21]:
# Посмотрим на лучшие параметры и лучшую оценку

print(search.best_params_)
print(search.best_score_)

{'alpha': 0.5, 'random_state': 42, 'solver': 'sag', 'tol': 0.01}
-1.3645594120400553


Лучшее качество для линейной модели с L2 регуляризацией - 1.3645594120400553

Теперь поперебираем параметры для нейронной сети из sklearn

In [ ]:
nn_config = {
    
}